# Pothole Detection — YOLOv8 Quickstart
This notebook trains a **pothole detector** with **YOLOv8**.
It expects data in YOLO format with a single class `pothole`. See README for details.

In [ ]:

# Install deps (uncomment if running locally)
# !pip install -r ../requirements.txt

from ultralytics import YOLO
import os, json, glob, shutil, random
from pathlib import Path
print("Ultralytics version:", YOLO.__version__)


## 1) (Optional) Convert Pascal VOC XML to YOLO
If your dataset uses VOC XML, run the converter. Adjust paths as needed.

In [ ]:

# Example: convert VOC to YOLO (edit the paths before running)
# %run ../src/voc_to_yolo.py --images /path/to/images --ann /path/to/xmls --out ../data --split 0.85 --only pothole --copy


## 2) Verify dataset layout
Expected: `../data/images/{train,val}` and `../data/labels/{train,val}`

In [ ]:

from pathlib import Path
root = Path("..")
for p in ["data/images/train","data/images/val","data/labels/train","data/labels/val"]:
    pth = root/p
    print(pth, "exists?" , pth.exists())


## 3) Train YOLOv8 (detection)
Start with a small model (`yolov8n.pt`) and iterate.

In [ ]:

from ultralytics import YOLO

data_cfg = str((Path("..")/"configs"/"data.yaml").resolve())
model = YOLO("yolov8n.pt")  # nano model to start; try yolov8s.pt later
results = model.train(data=data_cfg, epochs=50, imgsz=1024, batch=8, device=0)
model.val()


## 4) Inference demo on a sample image

In [ ]:

import cv2, matplotlib.pyplot as plt

# Put a test image under ../data/images/val/ to visualize
val_imgs = list((Path("..")/"data"/"images"/"val").glob("*.*"))
if val_imgs:
    test_im = str(val_imgs[0])
    res = model.predict(source=test_im, conf=0.35, imgsz=1024)[0]
    im_bgr = res.plot()  # annotated
    im_rgb = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10,8))
    plt.imshow(im_rgb); plt.axis('off')
else:
    print("No images found under data/images/val")


## 5) Export for deployment (pick one)

In [ ]:

# ONNX
model.export(format="onnx")

# TensorRT (requires environment support)
# model.export(format="engine")

# CoreML (macOS / iOS dev)
# model.export(format="coreml")

# TFLite
# model.export(format="tflite")


## (Optional) Severity scoring with monocular depth (MiDaS)

In [ ]:

# !pip install timm

import torch, numpy as np, cv2, matplotlib.pyplot as plt
from pathlib import Path

# Load MiDaS
midas = torch.hub.load("intel-isl/MiDaS", "DPT_Hybrid")
transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = transforms.dpt_transform
midas.eval()

def mean_depth_inside_mask(depth_map, mask_xy):
    m = np.zeros(depth_map.shape, dtype=np.uint8)
    cv2.fillPoly(m, [mask_xy.astype(np.int32)], 1)
    vals = depth_map[m==1]
    return float(vals.mean()) if vals.size else 0.0

# Example: run on one prediction and compute mean depth within each box area (approximate using boxes)
val_imgs = list((Path("..")/"data"/"images"/"val").glob("*.*"))
if val_imgs:
    test_im = cv2.imread(str(val_imgs[0]))
    inp = transform(cv2.cvtColor(test_im, cv2.COLOR_BGR2RGB)).unsqueeze(0)
    with torch.no_grad():
        depth = midas(inp).squeeze().cpu().numpy()
    depth = (depth - depth.min()) / (depth.max() - depth.min() + 1e-6)

    res = model.predict(source=str(val_imgs[0]), conf=0.35, imgsz=1024)[0]
    scores = []
    for b in res.boxes.xyxy.cpu().numpy():
        x1,y1,x2,y2 = b.astype(int)
        mask_xy = np.array([[x1,y1],[x2,y1],[x2,y2],[x1,y2]])
        area = (x2-x1)*(y2-y1)
        md = mean_depth_inside_mask(depth, mask_xy)
        scores.append({"bbox": [int(x) for x in [x1,y1,x2,y2]], "area_px": int(area), "mean_depth": md})
    print(json.dumps(scores, indent=2))
else:
    print("Add a sample to data/images/val first.")
